# Análisis Explotario de Datos
- Dataset: Predict Students' Dropout and Academic Success
- Fuente: UCI Repository
- Autor:  --
- Fecha: Enero 2026

In [ ]:
import sys
import os
# Obtener ruta absoluta al proyecto (un nivel arriba de notebooks/)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Agregar al path
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import os
from src.data.clean_columns import clean_dataframe_columns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils.constants import (
    VARS_BINARIAS,
    VARS_CATEGORICAS_NOMINALES,
    VARS_CATEGORICAS_ORDINALES,
    VARS_NUMERICAS,
    TARGET,
    TARGET_VALUES,
    LABELS    
)



## 1. Carga y Dimensión del dataset
Este paso permite conocer cuántas observaciones y variables contiene el dataset.

In [ ]:
# Cargar el dataset
df = pd.read_csv('../data/raw/data.csv', delimiter=';')
df = clean_dataframe_columns(df)

# Verifica carga
print("================================================================================")
print("1. CARGA Y DIMENSIÓN DEL DATASET")
print("================================================================================")
print(f"\n Dataset cargado")
print(f" - Observaciones (filas): {df.shape[0]:,}")
print(f" - Variables (columnas): {df.shape[1]}" + ", incluye columna Target")
print(f" - Total de celdas: {df.shape[0] * df.shape[1]:,}")

# Primeras filas
print("\n" + "--------------------------------------------------------------------------------")
print("Primeras 5 filas del dataset:")
print("--------------------------------------------------------------------------------")
df.head()

## 2. Listado de variables del dataset, verificación de nulos y duplicados
Este paso permite conocer cuántas observaciones y variables contiene el dataset.

In [ ]:
print("================================================================================")
print("2. LISTADO DE VARIABLES DEL DATASET, VERIFICACION DE NULOS Y DUPLICADOS")
print("================================================================================")

# Lista las variables
print(f"\n{'#':<4} {'Variable':<55} {'Tipo':<10}")
print("--------------------------------------------------------------------------------")
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:<4} {col:<55} {str(dtype):<10}")

# Verificar valores nulos
print("\n" + "--------------------------------------------------------------------------------")
print(" Verificación de Valores Nulos:")
print("--------------------------------------------------------------------------------")
null_counts = df.isnull().sum()
total_nulls = null_counts.sum()

if total_nulls == 0:
    print(" No hay valores nulos en el dataset")
else:
    print(f" Total de valores nulos: {total_nulls}")
    print(null_counts[null_counts > 0])

# Verificar Duplicados
print("\n" + "--------------------------------------------------------------------------------")
print(" Verificación de Valores duplicados:")
print("--------------------------------------------------------------------------------")
duplicates = df.duplicated().sum()
if duplicates == 0:
    print(" No hay registros duplicados")
else:
    print(f" Registros duplicados: {duplicates}")
    

## 3. Análisis de variables
Se evalúa las variables por grupo de acuerdo al tipo numéricas, categóricas (nominales y ordinales)

### 3.0. Clasificación de variables por tipo

In [ ]:
print("================================================================================")
print("3.0. CLASIFICACIÓN DE VARIABLES POR TIPO")
print("================================================================================")

total_classified = (len(VARS_BINARIAS) + len(VARS_CATEGORICAS_NOMINALES) + 
                    len(VARS_CATEGORICAS_ORDINALES) + len(VARS_NUMERICAS) + 
                    len(TARGET))

print(f"\n CANTIDAD DE VARIABLES NUMÉRICAS: {len(VARS_NUMERICAS)} variables")
print(f"\n CANTIDAD DE VARIABLES CATEGÓRICAS BINARIAS: {len(VARS_BINARIAS)} variables")
print(f"\n CANTIDAD DE VARIABLES CATEGÓRICAS NOMINALES: {len(VARS_CATEGORICAS_NOMINALES)} variables")
print(f"\n CANTIDAD DE VARIABLES CATEGÓRICAS ORDINALES: {len(VARS_CATEGORICAS_ORDINALES)} variable")
print(f"\n TARGET: {len(TARGET)} variable (Clases: {', '.join(TARGET_VALUES)})")
print(f"\n TOTAL VARIABLES CLASIFICADAS: {total_classified}")

### 3.1. Análisis variables numéricas

#### 3.1.1. Análisis UNIVARIADO

Estadisticas 

In [ ]:
df[VARS_NUMERICAS].describe().T

Histogramas

In [ ]:
import warnings

# Ajustes globales
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (20, 20)
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["xtick.labelsize"] = 10
plt.rcParams["ytick.labelsize"] = 10
warnings.filterwarnings("ignore", message="Glyph.*missing")

OUTPUT_DIR = "../outputs/figures/EDA/1_numericas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"01_distribucion_variables_numericas.png"
filepath = os.path.join(OUTPUT_DIR, filename)

# Calcular número de filas necesarias para 5 columnas
n_vars = len(VARS_NUMERICAS)
n_cols = 5
n_rows = (n_vars // n_cols) + (1 if n_vars % n_cols != 0 else 0)

# Histograma de todas las columnas numéricas con 5 columnas
df[VARS_NUMERICAS].hist(bins=20, figsize=(20, 16), grid=False, layout=(n_rows, n_cols))
plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

Diagramas de cajas

In [ ]:
# Boxplots con Seaborn
n_vars = len(VARS_NUMERICAS)
n_cols = 5
n_rows = (n_vars // n_cols) + (1 if n_vars % n_cols != 0 else 0)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 14))
axes = axes.flatten()

OUTPUT_DIR = "../outputs/figures/EDA/1_numericas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"02_boxplot_variables_numericas.png"
filepath = os.path.join(OUTPUT_DIR, filename)

for i, col in enumerate(VARS_NUMERICAS):
    ax = axes[i]
    sns.boxplot(y=df[col], ax=ax, color='steelblue', width=0.5)
    ax.set_title(col, fontsize=10, fontweight='bold')
    ax.set_ylabel('')
    
    # Agregar información de outliers
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    n_outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    pct = (n_outliers / len(df)) * 100
    
    ax.annotate(f'n={n_outliers} ({pct:.1f}%)', xy=(0.95, 0.95), xycoords='axes fraction',
                ha='right', va='top', fontsize=8, color='black')

# Ocultar subplots vacíos
for j in range(len(VARS_NUMERICAS), len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

#### 3.1.2. Análisis BIVARIADO

Matriz de correlación

In [ ]:
# Calcular matriz de correlación
corr_matrix = df[VARS_NUMERICAS].corr()

# Visualización: Heatmap completo
fig, ax = plt.subplots(figsize=(14, 12))

mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5,
            cbar_kws={'shrink': 0.8, 'label': 'Correlación'},
            annot_kws={'size': 8}, ax=ax)

ax.set_title('Matriz de Correlación - Variables Numéricas', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(fontsize=9)

OUTPUT_DIR = "../outputs/figures/EDA/1_numericas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"03_correlation_matrix_variables_numericas.png"
filepath = os.path.join(OUTPUT_DIR, filename)

plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

Diagramas de caja - Variables numéricas versus target

In [ ]:
# Boxplots por Target (para ver diferencias entre clases)
n_vars = len(VARS_NUMERICAS)
n_cols = 5
n_rows = (n_vars // n_cols) + (1 if n_vars % n_cols != 0 else 0)

OUTPUT_DIR = "../outputs/figures/EDA/1_numericas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"04_boxplot_variables_numericas_by_target.png"
filepath = os.path.join(OUTPUT_DIR, filename)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 16))
axes = axes.flatten()

colors = {'Dropout': '#E74C3C', 'Enrolled': '#1f77b4', 'Graduate': '#2ca02c'}


for i, col in enumerate(VARS_NUMERICAS):
    ax = axes[i]
    sns.boxplot(x='target', y=col, data=df, ax=ax, 
                hue='target', palette=colors, width=0.6, legend=False)
    ax.set_title(col, fontsize=10, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='x', labelsize=8)

# Ocultar subplots vacíos
for j in range(len(VARS_NUMERICAS), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Distribución de Variables Numéricas por Target', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()


#### 3.1.3 Conclusion variables numércias

1. Distribuciones. Se debe evaluar potencial transformación de ser necesario de acuerdo al algoritmo.
- Las variables que presentan asimetrias positivas y sesgos extremos son:age_at_enrollment, curricular_units_1st_sem_credited, curricular_units_1st_sem_without_evaluations, curricular_units_2nd_sem_without_evaluations y curricular_units_2nd_sem_credited. Estas distribuciones sesgadas pueden requerir transformaciones (log, box-cox, power transform) dependiendo del algoritmo de modelado seleccionado.
- Las variables curricular_units_1st_sem_grade y curricular_units_2nd_sem_grade presentan una distribución bimodal con peak en 0 (no tiene calificaciones) y 12-14, existiendo dos poblaciones diferenciadas: estudiantes que no rinden evaluaciones y estudiantes activos con rendimiento normal.
- Las variables curricular_units_1st_sem_without_evaluations y curricular_units_2nd_sem_without_evaluations presentan una alta concetración en 0 (zero_inflated, pero es propio del conyexto), esto implica que la gran mayoria de los alumnos realiza los exámenes, siendo un signo de estuadiante activo, se debe evaluar transformación de ser necesario.
- Las variables curricular_units_1st_sem_credited y curricular_units_2nd_sem_credited presentan una alta concetración en 0  (zero_inflated, pero es propio del conyexto), esto implica que la gran mayoria de los alumnos no convaidan ramos curados previamente, siendo un signo de estuadiante activocursando desde cero el grado.
- Las variables previous_qualification_grade y admission_grade presentan una distribución normal, curricular_units_1st_sem_enrolled y curricular_units_2nd_sem_enrolled presentan una distribución aproximadamente normal. Estas variables son adecuadas para algoritmos que suponen normalidad, sin necesidad de transformaciones adicionales.

2. Outliers.
- Las siguiente variables presentan alta tasas de valores fuera de rango calculado con IQR de 1.5 estas variables presentan maás de un 10%: curricular_units_2nd_sem_grade: 19.8% (877 outliers), curricular_units_1st_sem_grade: 16.4% (726 outliers), curricular_units_1st_sem_credited: 13.0% (577 outliers), curricular_units_2nd_sem_credited: 12.0% (530 outliers) y age_at_enrollment: 10.0% (441 outliers). Estos valores atípicos no necesariamente serán eliminados, ya que representan comportamientos reales del alumnado (rendimientos extremos o inscripciones/creditaciones inusuales). Pueden ser tratados mediante winsorization o técnicas robustas según el modelo, se decidirá en etapa de preprocesamiento.

3. Variables macroeconómicas (unemployment_rate, inflation_rate, gdp).
- Sin outliers (por definición).                                                 
- 9-10 valores únicos.
- Estas variables presentan bajo poder predictivo y probablemente aportarán poca ganancia al modelo.

4. Correlaciones
- ALTA (r > 0.7) 
    - 1st_sem_credited y 2nd_sem_credited:     r = 0.94
    - 1st_sem_enrolled y 2nd_sem_enrolled:     r = 0.94
    - 1st_sem_enrolled y 2nd_sem_credited:     r = 0.75
    - 1st_sem_approved y 2nd_sem_approved:     r = 0.90
    - 1st_sem_approved y 1st_sem_evaluations:  r = 0.77
    - 2nd_sem_approved y 2nd_sem_evaluations:  r = 0.78
    - 1st_sem_grade  2nd_sem_grade:           r = 0.84
    - 1st_sem_grade y 1st_sem_approved:        r = 0.70
    - 2nd_sem_grade y 2nd_sem_approved:        r = 0.76
    - Estas correlaciones indican que el rendimiento académico entre semestres es altamente consistente, y puede requerir reducción de dimensionalidad (PCA o selección de features).

- ALTA (0.5 < r < 0.7) 
    - 1st_sem_enrolled y 1st_sem_evaluations:  r = 0.68
    - 2nd_sem_enrolled y 2nd_sem_evaluations:  r = 0.60
    - 1st_sem_approved y 2nd_sem_enrolled:     r = 0.61
    - previous_qual_grade y admission_grade:   r = 0.58
    - 1st_sem_without_eval y 2nd_sem_without:  r = 0.58

- BAJA (r < 0.3):
    - age_at_enrollment, baja correlación con casi todas
    - Variables macroeconómicas: casi sin correlación   

5. Análisis bivariado
- Las variables curricular_units_1st_sem_approved, curricular_units_2nd_sem_approved, curricular_units_1st_sem_grade y curricular_units_2nd_sem_grade poseen alto poder discriminativoy separan muy bien los grupos en lsa 3 clases.
- La variable age_at_enrollment es un buen predictor también, se conluye que a mayor edad (al momento de la matricula) mayor es el abandono.
- Las variables curricular_units_1st_sem_without_evaluations y curricular_units_2nd_sem_without_evaluations es una señal de abandono.
- La variable previous_qualification_grade y admission_grade tienen poder predictor pero leve ya que se presenta una diferencia leve en las medias de ambas variables.
- Las variables macroeconómicas (unemployment_rate, inflation_rate, gdp) no presentan diferencias visibles.
- El rendimiento en el primer semestre es el principal diferenciador entre clases del target
- La edad (age_at_enrollment) muestra leve relación: estudiantes mayores tienden levemente a mayor abandono.
- Patrones:
    - Estudiantes clasificados "Dropout", presentan menos aprobaciones en ambos semestres (curricular_units_1st_sem_approved y curricular_units_2nd_sem_approved), enores evaluaciones realizadas (curricular_units_1st_sem_evaluations y curricular_units_2nd_sem_evaluations), menores promedios de notas (curricular_units_1st_sem_grade y curricular_units_2nd_sem_grade), menores unidades inscritas (curricular_units_1st_sem_enrolled y curricular_units_2nd_sem_enrolled). 
    - Estudiantes clasificados "Graduate", presentan las mayores medianas en notas (curricular_units_1st_sem_grade y curricular_units_2nd_sem_grade), aprobaciones (curricular_units_1st_sem_approved y curricular_units_2nd_sem_approved) y cantidad de evaluaciones realizadas (curricular_units_2nd_sem_evaluations)
    - Estudiantes clasificados "Enrolled", se ubican en valores intermedios y tiene un comportamieno similar a Graduate pero con menor rendimiento general.

6. Conclusión

- El rendimiento académico temprano (primer semestre) es el principal predictor de la clasificación final del estudiante.
- Existen fuertes patrones de consistencia entre semestres, lo que sugiere que no todos los indicadores son necesarios, dado la multicolinealidad.
- Las variables con asimetría marcada y outliers deben considerarse para transformación, según el algoritmo elegido.
- Las variables macroeconómicas tienen poca influencia en el comportamiento estudiantil, mientras que los indicadores de participación académica y notas sí son altamente relevantes.
- La edad y la ausencia de evaluaciones aparecen como señales secundarias pero útiles para predecir riesgo de abandono.


### 3.2. Análisis variables binarias

#### 3.2.1. Análisis UNIVARIADO

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import math
import matplotlib.pyplot as plt
import seaborn as sns




# Crear DataFrame resumen
resume_vars = []


for var in VARS_BINARIAS:
    counts = df[var].value_counts().sort_index()
    n_0 = counts.get(0, 0)
    n_1 = counts.get(1, 0)
    pct_0 = (n_0 / len(df)) * 100
    pct_1 = (n_1 / len(df)) * 100
    
    labels = LABELS.get(var, {0: 'No', 1: 'Sí'})
    
    resume_vars.append({
        'Variable': var,
        'Label_0': labels[0],
        'N_0': n_0,
        '%_0': pct_0,
        'Label_1': labels[1],
        'N_1': n_1,
        '%_1': pct_1
    })

binary_df = pd.DataFrame(resume_vars)

print("\nDistribución de Variables Binarias:")
print("------------------------------------------------------------------------------------------------")
print(f"{'Variable':<30} {'Valor=0':<15} {'N':>7} {'%':>7}   {'Valor=1':<15} {'N':>7} {'%':>7}")
print("------------------------------------------------------------------------------------------------")

for _, row in binary_df.iterrows():
    print(f"{row['Variable']:<30} {row['Label_0']:<15} {row['N_0']:>7,} {row['%_0']:>6.1f}%   {row['Label_1']:<15} {row['N_1']:>7,} {row['%_1']:>6.1f}%")



def plot_categorical_univariate(df, cat_vars, n_cols=3):

    OUTPUT_DIR = "../outputs/figures/EDA/2_binarias/"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    # Nombre archivo 
    filename = f"05_Grafico_barras_variables_binarias.png"
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    n_vars = len(cat_vars)
    n_rows = math.ceil(n_vars / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4 * n_rows))
    axes = axes.flatten()

    for i, col in enumerate(cat_vars):
        ax = axes[i]

        counts = df[col].value_counts().sort_index()

        sns.barplot(x=counts.index.astype(str), y=counts.values, ax=ax, palette="Blues_r")

        ax.set_title(f"{col}", fontsize=11, fontweight='bold')
        ax.set_ylabel("Frecuencia")
        ax.set_xlabel("")
        
    # Ocultar subplots vacíos
    for j in range(n_vars, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()


# EJECUCIÓN
plot_categorical_univariate(df, VARS_BINARIAS, n_cols=3)

#### 3.2.2. Análisis BIVARIADO

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# Paleta consistente para todas las clases
palette = {
    "Dropout":"#E74C3C", 
    "Enrolled": "#1f77b4", 
    "Graduate": "#2ca02c"
}

def plot_categorical_bivariate(df, cat_vars, target="target", n_cols=3):

    OUTPUT_DIR = "../outputs/figures/EDA/2_binarias/"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    # Nombre archivo 
    filename = f"06_Grafico_variables_binarias_by_target.png"
    filepath = os.path.join(OUTPUT_DIR, filename)

    n_vars = len(cat_vars)
    n_rows = math.ceil(n_vars / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4 * n_rows))
    axes = axes.flatten()

    for i, col in enumerate(cat_vars):
        ax = axes[i]

        # Tabla cruzada normalizada
        ctab = pd.crosstab(df[col], df[target], normalize='index')

        # Aplicar colores en el orden correcto de columnas
        ctab.plot(
            kind='bar',
            stacked=True,
            ax=ax,
            color=[palette[c] for c in ctab.columns]  # 👈 colores personalizados
        )

        ax.set_title(f"{col} vs {target}", fontsize=11, fontweight='bold')
        ax.set_ylabel("Proporción")
        ax.set_xlabel("")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')

    # Ocultar gráficos sobrantes
    for j in range(n_vars, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()

# EJECUCIÓN
plot_categorical_bivariate(df, VARS_BINARIAS)

#### 3.2.3 Conclusion preliminar EDA variables binarias
1. Variable Daytime/evening attendance (0 = nocturno, 1 = diurno). La mayoría (~90%) pertenece al turno diurno (1) y presentando una menor tasa de deserción. El horario de clases sí tiene relación con el desempeño, pero no es un predictor fuerte del abandono.
2. Variable Displaced (0 = no, 1 = sí). Se visualiza un equilibrio a nivel de categoria desplazado y no desplazado y una leve alza de la proporción de deserción en los no desplazados. No genera diferencias fuertes entre clases de target, en consecuencia, es una variable poco discriminativa.
3. Variable educational_special_needs (0 = no, 1 = sí). Casi todos los estudiantes están en 0 (sin necesidades especiales) y respecto de la relación con el target no hay diferencia al menos visualmente, no aporta información de utilidad. 
4. Variable debtor (0 = sin deudas, 1 = deudor). La mayoría (~90%) no presenta deuda y en los deudores la proporción de deserción es altísima (≈ 65%), es una de las variables más predictivas de abandono.
5. Variable tuition_fees_up_to_date (0 = no al día, 1 = al día). La mayoría (–90%) tiene su matrícula al día presenta, y de quienes no están al día la proporción de deserción es altísima (~90%%).Es, junto con debtor, la variable más predictiva del modelo.
6. Variable gender (0 = mujer, 1 = hombre). En los estudiantes hay más mujeres que hombres y estos último presentan una mayor tasa de deserción.
7. Variable scholarship_holder (0 = no, 1 = sí). La mayoría (~75%) no posee beca de estudios y el grupo que si presenta beca tiene una menor tasa de deserción. en consecuencia, funciona como factor protector contra abandono
8. Variable international (0 = nacional, 1 = internacional). Casi todos son nacionales (0) y la proporción por target es similar. Variable con poca fuerza predictiva.
9. Hallazgos principales:
    - Variables con mayor capacidad de predicción: debtor, tuition_fees_up_to_date y tuition_fees_up_to_date
    - Variables moderamenten informativas: daytimeevening_attendance y scholarship_holder
    - Variables poco relevantes: displaced, educational_special_needs y international

### 3.3. Análisis variables categóricas ordinales y nominales

#### 3.3.1. Análisis UNIVARIADO

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

OUTPUT_DIR = "../outputs/figures/EDA/3_categoricas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)



def plot_categorico_univariado(df, column):
    plt.figure(figsize=(10,4))
    
    df[column].value_counts().plot(kind='bar')
    
    plt.title(f"Distribución de {column}")
    plt.xlabel(column)
    plt.ylabel("Frecuencia")
    plt.xticks(rotation=45)
    plt.tight_layout()

    filename = f"07_univariado_{column}.png"
    filepath = os.path.join(OUTPUT_DIR, filename)
    # Guardar imagen
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()

    # =================================================================
    # TABLA CON PROPORCIÓN
    # =================================================================
   
    # Crear tabla resumen
    summary = pd.DataFrame({
        'N': df[column].value_counts(),
        'Porcentaje (%)': (df[column].value_counts(normalize=True) * 100).round(2)
    })

    # Agregar descripción
    if column in LABELS:
        summary['Descripcion'] = summary.index.map(LABELS[column])
        # Reordenar columnas para que Descripcion esté primero
        summary = summary[['Descripcion', 'N', 'Porcentaje (%)']]
        
    # Ordenar por N descendente
    summary = summary.sort_values('N', ascending=False)
    display(summary)

VARS_CATEGORICAS = VARS_CATEGORICAS_NOMINALES + VARS_CATEGORICAS_ORDINALES

# Ejecutar para todas las variables categóricas
for col in VARS_CATEGORICAS:
    print(f"\n ---- Variable: {col.upper()} ----\n")
    plot_categorico_univariado(df, col)

##### 3.3.1.1 Análisis UNIVARIADO - Conclusiones preliminares

1. Variable marital_status (6 categorías). Las 2 categorías más representativas son 1 (soltero) con un 89% y 2 (casado) con un 9%. Las demás categorías tiene un porcentaje menor al 3%, no representativo. Esto podría implicar agrupación de categorías y binarizar categorías resultantes (de ser 2). Se debe continuar con el análisis bivariado para analizar la variabilidad respecto del target.

2. Variable application_mode(18 categorias). Las 3 categorías más representativas (76% del dataset) son 1 (1ra fase - contingente general) con un 39%, 17 (2da fase - contingente general) con un 20% y 39 (Mayor de 23 años) con un 18%. Las demás categorías tiene un porcentaje menor al 8%, no representativo.

3. Variable course (17 categorías). Relativamente balanceadas (todas mayor a 100 registros) y solo se visualiza un caso atípico (33, 'Biofuel Production Technologies' con 12 casos), las 3 categorías más representativas son 9500 (Nursing) con un 17%, 9147 (Management) con un 9%, 9085 (Social Service) con un 8% y 9773 (Journalism and Communication) con un 7%.

4. Variable nacionality. (21 categorias). La nacionalidad 1 (Portugués) representa el 98% de los estudiantes, 19 nacionalidades restantes no superan los 20 registros, la gran mayoria tiene menos de 5, a nivel estadístico no aportará información al modelo. 

5. Variable previous_qualification (17 categorías). La categoría 1 (Educación secundaria) domina con un 84% y 16% tiene otra cualificación, de estas ultimas 7 categorías presentan
menos de 10 registros para tener en cuenta en una eventual agrupación. Para 39 (Especialización tecnológica), 19 (Ed. básica 3er ciclo) y 3 (Ed. superior - grado) presentan más de 100 registros.

6. Variable fathers_qualification(34 categorías). Top 4 categorías = 85.5% del dataset: 37 (Ed. básica 1er ciclo) 27%, 19 (Ed. básica 3er ciclo) 22%, 1 (Secundaria (12º año)) 20%,
38 (Ed. básica 2do ciclo) 16%.  22 categorías con N < 10 registros y 6 de estas con N=1.  Educación de padres alcanza un 65% de educación básica (37 + 19 + 38) y secundaria completa un  20% (1). Se detecta 5 categorías (13, 20, 25 ,31, 33) con valores nulos. Requiere agrupación obligatoria 

7. Variable mothers_qualification (29 categorías, 5 menos que fathers). Top 5 categorías = 91% del dataset:  1 (Secundaria (12º año)) 24%, 37 (Ed. básica 1er ciclo) 23%, 19 (Ed. básica 3er ciclo) 22%, 38 (Ed. básica 2do ciclo) 13% y 3(Superior - Grado) 10%. 19 categorías con N < 10 registros y 5 de estas con N=1.  Educación de madres alcanza un 57% de educación básica (37 + 19 + 38) y secundaria completa un 24.2% (mayor que los padres (20%)), más balanceada que fathers_qualification y requiere agrupación obligatoria. 

8. Variable mothers_occupation (32 categorías). Top 3 categorías = 66% del dataset: 9 (Trabajadores no cualificados) 36%, 4 (Personal administrativo) 18%, 5 (Servicios/Seguridad/Vendedores) 12%. 17 categorías con N < 10 registros y 4 de estas con N=1. Requiere agrupación obligatoria.

9. Variable fathers_occupation (46 categorías). Top 3 categorías = 46% del dataset: 9 (Trabajadores no cualificados) 23%, 7 (Trabajadores industria/construcción) 15%, 5 (Servicios/Seguridad/Vendedores) 12%. 32 categorías con N < 10 registros y 14 de estas con N=1. Requiere agrupación obligatoria.

10. Variable application_order (ordinal con 9 categorías). Presenta una distribución sesgada con dominancia de 1 categoría, dado que es ordinal a mayor código menor es la preferencia del estudiante. Top 2 categorías = 80% del dataset: 1 (2da opción) 68%, 2 (3ra opción) 12%. Un registro atípico es que el código 0 (1ra opción), solo hay 1 estudiante y con 9 (Última opción) sucede lo mismo.

#### 3.3.2. Análisis BIVARIADO

In [ ]:
import numpy as np
import os

# Paleta para el target
palette_target = {
    "Dropout": "#E74C3C",   # Rojo
    "Enrolled": "#1F77B4",  # Azul
    "Graduate": "#2CA02C"   # Verde
}



OUTPUT_DIR = "../outputs/figures/EDA/3_categoricas/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def plot_categorico_bivariado(df, column, target="target"):
    ctab = pd.crosstab(df[column], df[target], normalize="index")

    print(f"\n===== {column.upper()} vs TARGET =====")
    
    # Decidir orientación según número de categorías
    if len(ctab) > 8:
        figsize = (10, 6)
        kind = "barh"
    else:
        figsize = (8, 4)
        kind = "bar"

    fig, ax = plt.subplots(figsize=figsize)

    ctab.plot(
        kind=kind,
        stacked=True,
        color=list(palette_target.values()),
        ax=ax,
        edgecolor="black"
    )

    plt.title(f"{column} vs {target} (proporciones)", fontsize=12)

    # Ajustar etiquetas según orientación
    if kind == "bar":
        ax.set_xlabel(column)
        ax.set_ylabel("Proporción")
        plt.xticks(rotation=25, ha="right", fontsize=8)
    else:
        ax.set_xlabel("Proporción")  # En horizontal, X es la proporción
        ax.set_ylabel(column)         # Y es la variable categórica
        plt.yticks(fontsize=8)

    # Leyenda fuera del gráfico
    ax.legend(title="Target", bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.tight_layout()

    filename = f"08_bivariado_{column}.png"
    filepath = os.path.join(OUTPUT_DIR, filename)
    # Guardar imagen
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()

    # =================================================================
    # TABLA CON PROPORCIÓN + DISTRIBUCIÓN POR TARGET
    # =================================================================
    
    # Crosstab con conteos
    ct_counts = pd.crosstab(df[column], df['target'])
    
    # Crosstab con porcentajes por fila (cada categoría suma 100%)
    ct_pct = pd.crosstab(df[column], df['target'], normalize='index') * 100

    # Crear tabla resumen
    summary = pd.DataFrame({
        'N': df[column].value_counts(),
        'Porcentaje': df[column].value_counts(normalize=True).round(2),
        'Dropout_N': ct_counts['Dropout'],
        'Dropout_%': ct_pct['Dropout'].round(2),
        'Enrolled_N': ct_counts['Enrolled'],
        'Enrolled_%': ct_pct['Enrolled'].round(2),
        'Graduate_N': ct_counts['Graduate'],
        'Graduate_%': ct_pct['Graduate'].round(2)
    })

    # Agregar descripción
    if column in LABELS:
        summary['Descripcion'] = summary.index.map(LABELS[column])
        # Reordenar columnas para que Descripcion esté primero
        summary = summary[['Descripcion', 'N', 'Porcentaje','Dropout_N','Dropout_%','Enrolled_N','Enrolled_%','Graduate_N','Graduate_%']]
    
    # Ordenar por N descendente
    summary = summary.sort_values('N', ascending=False)
    
    display(summary)

# Ejecutar
for col in VARS_CATEGORICAS_NOMINALES + VARS_CATEGORICAS_ORDINALES:
    plot_categorico_bivariado(df, col)

##### 3.3.1.2 Análisis BIVARIADO - Conclusiones preliminares

1. Variable marital_status. De las 2 categorías más representativas 1 (soltero) desertan un 30% y 2 (casado) un 47%, este último grupo tiene 1.5 veces más riesgo de desertar y las restantes presentan muy pocos casos (menos de 30 registros) para generar variabilidad significativa respecto del target. Dado este escenario, se creará un nueva binaria 
'es_soltero' con valores 1 para solteros y 0 para no solteros.
 
2. Variable application_mode. De las 3 categorías más representativas  1 (1ra fase - contingente general), 17 (2da fase - contingente general) y 39 (Mayor de 23 años) con un 18%, la que presenta mayor % de deserción es 39 (Mayor de 23 años). Dado este escenario, se podrían realizar 2 tareas en etapa de preprocesamiento: 1-.agrupar las categorías por algún concepto y 2-. analizar la creación de una variable que permita la identificación de esta categoría.

3. Variable course. El rango de deserción por programa es muy amplio (15% - 67%), se considera que es una variable altamente predictiva. Nursing (950) presenta la menor tasa de deserción (15.4%) y tiene la mayor número de estudiantes (N=765). Esto sugiere alta motivación y/o mejor soporte en programas de salud. En cambio, los programas técnicos como Informatics Engineering (9119) 54.1% y Equinculture (9130) presentan las mayores tasas de deserción del 54% y 55%. Pre-procesamiento posible encoding. 

4. Variable nacionality. En la nacionalidad 1 (Portugués) desertán el 32.2% (4.314 estudiantes), todas las demás nacionalidades si bien presentan porcentajes de deserción los registros no son suficientes para ser considerados. En consecuencia, no hay diferencia significativa en deserción. Esta variable no se considerará en los modelos. 

5. Variable previous_qualification. Los estudiantes con Educación secundaria desertan en un 29%, con Especialización tecnológica un 32%, con Ed. básica 3er ciclo un 64%. Existe un escenario 
paradógico que los estudiantes con menor grado de estudios (9, 12, 10 y 15) y con estudios de post titulo (2, 3) presentan una tasa de deserción sobre el 50%. Se visualiza posible agrupación 
de categorias por riesgo de deserción, se debe evaluar como quedaría la distribución.

6. Variable fathers_qualification. La categoría 34 (desconocido) presenta la mayor de serción con un 72%, casos en que el estudiantes que no reportan información del padre. Con un padre con 37 (Ed. básica 1er ciclo) y 38 (Ed. básica 2do ciclo) se tienen tasas de deserción del 36% y 24%.  No hay relación lineal clara entre nivel educativo y deserción.

7. Variable mothers_qualification. La categoría 34 (desconocido) presenta la mayor de serción con un 74%, casos en que el estudiantes que no reportan info de la madre. Con un padre con 37 (Ed. básica 1er ciclo) y 38 (Ed. básica 2do ciclo) se tienen tasas de deserción del 38% y 25%.  No hay relación lineal clara entre nivel educativo y deserción 

8. Variable mothers_occupation. Las categorías que presentan mayor deserción son: 99 ("en blanco") 77%, 90 (Otra situación) 73%, 0 (Estudiante) 68%. En este ultimo caso una madre estudiante podría implicar una situación económica inestable. Las demás ocupaciones tienen tasas similares de deserción con un rango entre 27% - 32% no existiendo gran diferencia entre ocupaciones específicas.

9. Variable fathers_occupation. Presenta un comportamiento símil a  la variable mothers_occupation Las categorias que presentan mayor deserción son: 99 ("en blanco") 68%, 90 (Otra situación) 71%, 0 (Estudiante) 64%. En este ultimo caso un padre estudiante podría implicar una situación económica inestable. Las demás ocupaciones tienen tasas similares de deserción con un rango entre 28% - 36% no existiendo gran diferencia entre ocupaciones específicas.

10. Variable application_order. Se detecta patrón especial, dado que el mayor indice de deserción se encuentra en las primeras opciones de postulacion al grado, por ejemplo, estudiantes que eligieron como 2da opción (mayoría) tienen la mayor deserción (34.8%) y estudiantes que eligieron como 7ma opción tienen la menor deserción (22.6%).

#### 3.3.3 Conclusiones análisis variables categóricas ordinales y nominales

1. Algunas variables categóricas son altamente predictivas, especialmente:
- course (variación extrema en deserción 15–67%)
- application_mode (categoría >23 años = muy riesgosa)
- fathers_qualification y mothers_qualification sin información de (≈ 70% deserción)

2. Otras variables presentan bajo valor predictivo o alta concentración en una categoría, será evaluada su exclusión:
- nacionality
- educational_special_needs
- international

3. Muchas variables necesitan agrupación o ingeniería de caracteristicas:
- Agrupación de estudios previos (ej. riesgo_calificacion_previa)
- Agrupación de ocupación (ej. nivel_ocupacion)
- Variable derivada: has_unknown_parent_info 

4. La variable ordinal application_order no presenta un patrón lineal, pero sigue aportando información moderada sobre preferencia del estudiante y riesgo de abandono.

5. Las variables categóricas nominales y la ordinal del dataset aportan información estructural relevante sobre antecedentes académicos, preferencias de postulación y contexto socioeconómico. Varias de ellas, especialmente course, application_mode, y atributos familiares, muestran un papel significativo en la predicción del abandono. Sin embargo, otras variables, debido a su fuerte desbalance o baja sensibilidad al fenómeno de deserción, deben ser excluidas o transformadas. El adecuado tratamiento de estas variables mediante agrupaciones, encoding avanzado y creación de variables derivadas permitirá mejorar sustancialmente el rendimiento de los modelos predictivos.

### 3.4 Distribución de la variable objetivo `Target`
Valores:
- 0 = Dropout
- 1 = Graduate
- 2 = Enrolled

In [ ]:
plt.figure(figsize=(5, 4))   # <-- más pequeño
df['target'].value_counts(normalize=True).plot(
    kind='bar',
    #color=['red', 'green', 'blue']
    color=['#E74C3C', '#2ca02c', '#1f77b4']
)


OUTPUT_DIR = "../outputs/figures/EDA/4_target/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"09_distribucion_target.png"
filepath = os.path.join(OUTPUT_DIR, filename)

plt.title("Distribución de la variable objetivo", fontsize=12)
plt.xlabel("Clase", fontsize=10)
plt.ylabel("Proporción", fontsize=10)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

df['target'].value_counts()

#### Conclusión de Distribución de la variable objetivo `Target`

1. La variable objetivo target presenta tres posibles resultados académicos para cada estudiante: Graduate, Dropout y Enrolled. La distribución observada en el dataset es la siguiente:
- Graduate: 2.209 estudiantes (≈ 50%). Corresponde a estudiantes que lograron titularse (Graduate), representando aproximadamente la mitad del total. 
- Dropout: 1.421 estudiantes (≈ 32%). La tasa de deserción (Dropout) es notablemente alta, alcanzando un 32%, lo que confirma que la deserción es un fenómeno relevante y suficientemente frecuente como para justificar el desarrollo de modelos predictivos e intervenciones institucionales.
- Enrolled: 794 estudiantes (≈ 18%). Corresponde a estudiantes que permanecen activos o no han completado su trayectoria. Aunque este grupo no representa abandono directo, y continua con un grado en proceso.

2. La relación entre clases es aproximadamente 2:1 entre No Dropout (Graduate + Enrolled) y Dropout y es un desbalance moderado.

3. La distribución del target evidencia que la deserción afecta a uno de cada tres estudiantes, lo que convierte a este problema en una prioridad institucional. La distribución es adecuada para el modelado predictivo y proporciona una base sólida para explorar factores asociados al abandono académico mediante el Análisis Exploratorio de Datos y posteriormente los modelos de Machine Learning.
 



## 4. Conclusiones del EDA

- El dataset contiene **4.424 registros y 37 variables**, con buena estructura.
- No se observan valores nulos significativos.
- Existe un **marcado desbalance** en la variable objetivo.
- Las variables numéricas muestran patrones esperables, aunque con algunos valores extremos.
- Se identifican **correlaciones importantes** entre variables de rendimiento.
- Las variables categóricas están codificadas numéricamente; requerirán transformaciones.
- El dataset es apropiado para modelos supervisados, pero se necesitará posiblemente:
  - Ingeniería de características
  - Normalización/estandarización
  - Manejo del desbalance (SMOTE o class weights)
  - Selección correcta de la variable objetivo (binaria vs multiclase)

El EDA sienta las bases para avanzar hacia la fase de preprocesamiento y creación de pipelines.
